# Bayes Classification For Sentiment Analysis

### Using 5 Classes

In [ ]:
import pandas as pd
df = pd.read_csv("Corona_NLP_train.csv", encoding='latin-1')
T= pd.read_csv("Corona_NLP_test.csv")
df.Sentiment.unique()

In [ ]:
import re
def remove_urls(text): # to remove HTTP links
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
df['OriginalTweet']=df['OriginalTweet'].apply(lambda x:remove_urls(x))
T['OriginalTweet']=T['OriginalTweet'].apply(lambda x:remove_urls(x))
df1['OriginalTweet'].str.replace('http.*.com', '',regex = True)


In [ ]:
def remove_urls(text): # to remove HTML tags
    return re.sub(r'<.*?>', '', text)
df['OriginalTweet']=df['OriginalTweet'].apply(lambda x:remove_urls(x))
T['OriginalTweet']=T['OriginalTweet'].apply(lambda x:remove_urls(x))

In [ ]:
# Converting Clases
df['Sentiment']=df.Sentiment.replace('Neutral',0)
T['Sentiment']=T.Sentiment.replace('Neutral',0)
df['Sentiment']=df.Sentiment.replace('Positive',1)
T['Sentiment']=T.Sentiment.replace('Positive',1)
df['Sentiment']=df.Sentiment.replace('Negative',-1)
T['Sentiment']=T.Sentiment.replace('Negative',-1)
df['Sentiment']=df.Sentiment.replace('Extremely Negative',-2)
T['Sentiment']=T.Sentiment.replace('Extremely Negative',-2)
df['Sentiment']=df.Sentiment.replace('Extremely Positive',2)
T['Sentiment']=T.Sentiment.replace('Extremely Positive',2)

In [ ]:
reviews = df[["OriginalTweet","Sentiment"]].values
test = T[["OriginalTweet","Sentiment"]].values

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

# Generate counts from text using a vectorizer.  
# This performs our step of computing word counts.
vectorizer = CountVectorizer(stop_words='english')
train_features = vectorizer.fit_transform([r[0] for r in reviews])
test_features = vectorizer.transform([r[0] for r in test])

# Fit a naive bayes model to the training data.
# This will train the model using the word counts we computer, 
#and the existing classifications in the training set.
nb = MultinomialNB()
nb.fit(train_features, [int(r[1]) for r in reviews])

# predict classifications for our test features.
predictions = nb.predict(test_features)

In [ ]:
actual =T["Sentiment"].values
# Compute the error.
fpr, tpr, thresholds = metrics.roc_curve(actual, predictions, pos_label=1)
print("Multinomial naive bayes AUC: {0}".format(metrics.auc(fpr, tpr)))

Multinomial naive bayes AUC: 0.6225657867689026


In [ ]:
# confusion matrix to understand where is the 
# classifier wrongly clasifying
from sklearn.metrics import confusion_matrix
c=confusion_matrix(actual, predictions, labels=[2, 1, 0,-1,-2])
c

array([[174, 366,   5,  49,   5],
       [ 54, 624,  27, 214,  28],
       [ 14, 275, 138, 179,  13],
       [ 11, 361,  50, 559,  60],
       [  1,  80,   6, 340, 165]])

there is lot ofconfusion between **"Extremely Positive"** and **"Positive"**
& **"Extremely Negative"** and **"Negative"**

In [ ]:
sum=0
for i in range(5):
  for j in range(5):
    if i != j:
      sum = sum + c[i][j]

In [ ]:
sum/len(actual) # fraction of wrongly classified rows

0.5629278567667193

So Combining these classes increases the accuracy

### Using 3 classes

In [ ]:
df = pd.read_csv("Corona_NLP_train.csv", encoding='latin-1')
T= pd.read_csv("Corona_NLP_test.csv")

In [ ]:
import re
def remove_urls(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
df['OriginalTweet']=df['OriginalTweet'].apply(lambda x:remove_urls(x))
T['OriginalTweet']=T['OriginalTweet'].apply(lambda x:remove_urls(x))

In [ ]:
def remove_urls(text):
    return re.sub(r'<.*?>', '', text)
df['OriginalTweet']=df['OriginalTweet'].apply(lambda x:remove_urls(x))
T['OriginalTweet']=T['OriginalTweet'].apply(lambda x:remove_urls(x))

In [ ]:
# Converting classes
df['Sentiment']=df.Sentiment.replace('Neutral',0)
T['Sentiment']=T.Sentiment.replace('Neutral',0)
df['Sentiment']=df.Sentiment.replace('Positive',1)
T['Sentiment']=T.Sentiment.replace('Positive',1)
df['Sentiment']=df.Sentiment.replace('Negative',-1)
T['Sentiment']=T.Sentiment.replace('Negative',-1)
df['Sentiment']=df.Sentiment.replace('Extremely Negative',-1)
T['Sentiment']=T.Sentiment.replace('Extremely Negative',-1)
df['Sentiment']=df.Sentiment.replace('Extremely Positive',1)
T['Sentiment']=T.Sentiment.replace('Extremely Positive',1)

here there are only 3 classes 1,0,-1 made by combining **'Extremely Positive','Positive'** and **'Extremely Negative','Negative'**.

In [ ]:
reviews = df[["OriginalTweet","Sentiment"]].values
test = T[["OriginalTweet","Sentiment"]].values

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

# Generate counts from text using a vectorizer.  There are other vectorizers available, and lots of options you can set.
# This performs our step of computing word counts.
vectorizer = CountVectorizer(stop_words='english')
train_features = vectorizer.fit_transform([r[0] for r in reviews])
test_features = vectorizer.transform([r[0] for r in test])

# Fit a naive bayes model to the training data.
# This will train the model using the word counts we computer, and the existing classifications in the training set.
nb = MultinomialNB()
nb.fit(train_features, [int(r[1]) for r in reviews])

# Now we can use the model to predict classifications for our test features.
predictions = nb.predict(test_features)

In [ ]:
actual =T["Sentiment"].values
# Compute the error.  It is slightly different from our model because the internals of this process work differently from our implementation.
fpr, tpr, thresholds = metrics.roc_curve(actual, predictions, pos_label=1)
print("Multinomial naive bayes AUC: {0}".format(metrics.auc(fpr, tpr)))

Multinomial naive bayes AUC: 0.7385592567997629


In [ ]:
# confusion matrix to understand where is the 
# classifier wrongly clasifying
from sklearn.metrics import confusion_matrix
c=confusion_matrix(actual, predictions, labels=[1, 0,-1])
c

array([[1213,   16,  317],
       [ 307,   88,  224],
       [ 375,   36, 1222]])

True Positives have increased


In [ ]:
sum=0
for i in range(3):
  for j in range(3):
    if i != j:
      sum = sum + c[i][j]

In [ ]:
sum/len(actual)# fraction of wrongly classified rows

0.33570300157977884

This has dicreased from previous version